# OSSWatcher

# Role

## fetch github trending

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup

async def fetch_html(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()

async def parse_github_trending(html):
    soup = BeautifulSoup(html, 'html.parser')

    repositories = []

    for article in soup.select('article.Box-row'):
        repo_info = {}
        
        repo_info['name'] = article.select_one('h2 a').text.strip()
        repo_info['url'] = article.select_one('h2 a')['href'].strip()

        # Description
        description_element = article.select_one('p')
        repo_info['description'] = description_element.text.strip() if description_element else None

        # Language
        language_element = article.select_one('span[itemprop="programmingLanguage"]')
        repo_info['language'] = language_element.text.strip() if language_element else None

        # Stars and Forks
        stars_element = article.select('a.Link--muted')[0]
        forks_element = article.select('a.Link--muted')[1]
        repo_info['stars'] = stars_element.text.strip()
        repo_info['forks'] = forks_element.text.strip()

        # Today's Stars
        today_stars_element = article.select_one('span.d-inline-block.float-sm-right')
        repo_info['today_stars'] = today_stars_element.text.strip() if today_stars_element else None

        repositories.append(repo_info)

    return repositories

async def main():
    url = 'https://github.com/trending'
    html = await fetch_html(url)
    repositories = await parse_github_trending(html)

    for repo in repositories:
        print(f"Name: {repo['name']}")
        print(f"URL: https://github.com{repo['url']}")
        print(f"Description: {repo['description']}")
        print(f"Language: {repo['language']}")
        print(f"Stars: {repo['stars']}")
        print(f"Forks: {repo['forks']}")
        print(f"Today's Stars: {repo['today_stars']}")
        print()

In [2]:
await main()

Name: maybe-finance /

      maybe
URL: https://github.com/maybe-finance/maybe
Description: Personal finance and wealth management app
Language: TypeScript
Stars: 3,980
Forks: 273
Today's Stars: 1,320 stars today

Name: excalidraw /

      excalidraw
URL: https://github.com/excalidraw/excalidraw
Description: Virtual whiteboard for sketching hand-drawn like diagrams
Language: TypeScript
Stars: 64,083
Forks: 5,703
Today's Stars: 268 stars today

Name: atuinsh /

      atuin
URL: https://github.com/atuinsh/atuin
Description: ✨ Magical shell history
Language: Rust
Stars: 14,472
Forks: 395
Today's Stars: 248 stars today

Name: pandora-next /

      deploy
URL: https://github.com/pandora-next/deploy
Description: Pandora Cloud + Pandora Server + Shared Chat + BackendAPI Proxy + Chat2API + Signup Free = PandoraNext. New GPTs(Gizmo) UI, All in one!
Language: PHP
Stars: 9,706
Forks: 1,869
Today's Stars: 115 stars today

Name: xiaolai /

      everyone-can-use-english
URL: https://github.com/xiao

## Action: craw oss trending

In [15]:
import aiohttp
from bs4 import BeautifulSoup
from metagpt.actions.action import Action
from metagpt.config import CONFIG

class CrawlOSSTrending(Action):

    async def run(self, url: str = "https://github.com/trending"):
        async with aiohttp.ClientSession() as client:
            async with client.get(url, proxy=CONFIG.global_proxy) as response:
                response.raise_for_status()
                html = await response.text()
 
        soup = BeautifulSoup(html, 'html.parser')
    
        repositories = []
    
        for article in soup.select('article.Box-row'):
            repo_info = {}
            
            repo_info['name'] = article.select_one('h2 a').text.strip().replace("\n", "").replace(" ", "")
            repo_info['url'] = "https://github.com" + article.select_one('h2 a')['href'].strip()
    
            # Description
            description_element = article.select_one('p')
            repo_info['description'] = description_element.text.strip() if description_element else None
    
            # Language
            language_element = article.select_one('span[itemprop="programmingLanguage"]')
            repo_info['language'] = language_element.text.strip() if language_element else None
    
            # Stars and Forks
            stars_element = article.select('a.Link--muted')[0]
            forks_element = article.select('a.Link--muted')[1]
            repo_info['stars'] = stars_element.text.strip()
            repo_info['forks'] = forks_element.text.strip()
    
            # Today's Stars
            today_stars_element = article.select_one('span.d-inline-block.float-sm-right')
            repo_info['today_stars'] = today_stars_element.text.strip() if today_stars_element else None
    
            repositories.append(repo_info)
    
        return repositories

In [17]:
craw_oss_action = CrawlOSSTrending()
resp = await craw_oss_action.run()

In [18]:
resp

[{'name': 'EpicGames/raddebugger',
  'url': 'https://github.com/EpicGames/raddebugger',
  'description': 'A native, user-mode, multi-process, graphical debugger.',
  'language': 'C',
  'stars': '1,643',
  'forks': '68',
  'today_stars': '286 stars today'},
 {'name': 'danny-avila/LibreChat',
  'url': 'https://github.com/danny-avila/LibreChat',
  'description': 'Enhanced ChatGPT Clone: Features OpenAI, GPT-4 Vision, Bing, Anthropic, OpenRouter, Google Gemini, AI model switching, message search, langchain, DALL-E-3, ChatGPT Plugins, OpenAI Functions, Secure Multi-User System, Presets, completely open-source for self-hosting. More features in development',
  'language': 'TypeScript',
  'stars': '5,543',
  'forks': '1,026',
  'today_stars': '209 stars today'},
 {'name': 'xiaolai/everyone-can-use-english',
  'url': 'https://github.com/xiaolai/everyone-can-use-english',
  'description': '人人都能用英语',
  'language': 'TypeScript',
  'stars': '12,706',
  'forks': '2,467',
  'today_stars': '289 stars

## Action: summary the trending by llm

1. 今天榜单的整体趋势，例如哪几个编程语言比较热门、最热门的项目是哪些、主要集中在哪些领域
2. 榜单的仓库分类
3. 推荐进一步关注哪些仓库，推荐原因是什么

In [4]:
from typing import Any
from metagpt.actions.action import Action

TRENDING_ANALYSIS_PROMPT = """# Requirements
You are a GitHub Trending Analyst, aiming to provide users with insightful and personalized recommendations based on the latest
GitHub Trends. Based on the context, fill in the following missing information, generate engaging and informative titles, 
ensuring users discover repositories aligned with their interests.

# The title about Today's GitHub Trending
## Today's Trends: Uncover the Hottest GitHub Projects Today! Explore the trending programming languages and discover key domains capturing developers' attention. From ** to **, witness the top projects like never before.
## The Trends Categories: Dive into Today's GitHub Trending Domains! Explore featured projects in domains such as ** and **. Get a quick overview of each project, including programming languages, stars, and more.
## Highlights of the List: Spotlight noteworthy projects on GitHub Trending, including new tools, innovative projects, and rapidly gaining popularity, focusing on delivering distinctive and attention-grabbing content for users.
---
# Format Example

```
# [Title]

## Today's Trends
Today, ** and ** continue to dominate as the most popular programming languages. Key areas of interest include **, ** and **.
The top popular projects are Project1 and Project2.

## The Trends Categories
1. Generative AI
    - [Project1](https://github/xx/project1): [detail of the project, such as star total and today, language, ...]
    - [Project2](https://github/xx/project2): ...
...

## Highlights of the List
1. [Project1](https://github/xx/project1): [provide specific reasons why this project is recommended].
...
```

---
# Github Trending
{trending}
"""

class AnalysisOSSTrending(Action):

    async def run(
        self,
        trending: Any
    ):
        return await self._aask(TRENDING_ANALYSIS_PROMPT.format(trending=trending))

In [11]:
analysis_oss_trending = AnalysisOSSTrending()
await analysis_oss_trending.run(resp)

# Today's GitHub Trending: Discover the Latest and Hottest Projects!

## Today's Trends
Today, TypeScript and Rust continue to dominate as the most popular programming languages. Key areas of interest include personal finance management, virtual whiteboards, shell history, and cloud services. The top popular projects are [maybe-finance/maybe](https://github.com/maybe-finance/maybe) and [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw).

## The Trends Categories
1. Personal Finance Management
    - [maybe-finance/maybe](https://github.com/maybe-finance/maybe): A personal finance and wealth management app written in TypeScript, with 3,997 stars and 273 forks. It has gained 1,320 stars today.
2. Virtual Whiteboards
    - [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw): A virtual whiteboard for sketching hand-drawn like diagrams, written in TypeScript. It has 64,086 stars and 5,704 forks, with 268 new stars today.
3. Shell History
    - [atuinsh/atuin](htt

"# Today's GitHub Trending: Discover the Latest and Hottest Projects!\n\n## Today's Trends\nToday, TypeScript and Rust continue to dominate as the most popular programming languages. Key areas of interest include personal finance management, virtual whiteboards, shell history, and cloud services. The top popular projects are [maybe-finance/maybe](https://github.com/maybe-finance/maybe) and [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw).\n\n## The Trends Categories\n1. Personal Finance Management\n    - [maybe-finance/maybe](https://github.com/maybe-finance/maybe): A personal finance and wealth management app written in TypeScript, with 3,997 stars and 273 forks. It has gained 1,320 stars today.\n2. Virtual Whiteboards\n    - [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw): A virtual whiteboard for sketching hand-drawn like diagrams, written in TypeScript. It has 64,086 stars and 5,704 forks, with 268 new stars today.\n3. Shell History\n    - [atuins

## Role: OSSWatcher

In [5]:
# from metagpt.actions.oss_trending import CrawlOSSTrending, AnalysisOSSTrending
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger


class OssWatcher(Role):
    def __init__(
        self,
        name="Codey",
        profile="OssWatcher",
        goal="Generate an insightful GitHub Trending analysis report.",
        constraints="Only analyze based on the provided GitHub Trending data.",
    ):
        super().__init__(name, profile, goal, constraints)
        self._init_actions([CrawlOSSTrending, AnalysisOSSTrending])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self._rc.todo}")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self._rc.todo

        msg = self.get_memories(k=1)[0] # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=str(result), role=self.profile, cause_by=type(todo))
        self._rc.memory.add(msg)
        return msg

In [19]:
async def main():
    
    role = OssWatcher()
    result = await role.run("https://github.com/trending")
    logger.info(result)

await main()

2024-01-14 16:54:37.169 | INFO     | __main__:_act:20 - Codey(OssWatcher): ready to CrawlOSSTrending
2024-01-14 16:55:13.956 | INFO     | __main__:_act:20 - Codey(OssWatcher): ready to AnalysisOSSTrending


# Today's GitHub Trending Analysis

## Today's Trends
Today, TypeScript and Rust continue to dominate as the most popular programming languages. Key areas of interest include personal finance management, virtual whiteboard sketching, and magical shell history. The top popular projects are 'maybe-finance/maybe' and 'excalidraw/excalidraw'.

## The Trends Categories
1. Personal Finance Management
    - [maybe-finance/maybe](https://github.com/maybe-finance/maybe): A personal finance and wealth management app written in TypeScript, with 4,097 stars and 278 forks. It gained 1,320 stars today.
2. Virtual Whiteboard Sketching
    - [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw): A virtual whiteboard for sketching hand-drawn like diagrams, also written in TypeScript. It has 64,113 stars and 5,706 forks, with 268 new stars today.
3. Magical Shell History
    - [atuinsh/atuin](https://github.com/atuinsh/atuin): A magical shell history tool written in Rust, with 14,478 stars a

2024-01-14 16:55:27.959 | INFO     | __main__:main:5 - OssWatcher: # Today's GitHub Trending Analysis

## Today's Trends
Today, TypeScript and Rust continue to dominate as the most popular programming languages. Key areas of interest include personal finance management, virtual whiteboard sketching, and magical shell history. The top popular projects are 'maybe-finance/maybe' and 'excalidraw/excalidraw'.

## The Trends Categories
1. Personal Finance Management
    - [maybe-finance/maybe](https://github.com/maybe-finance/maybe): A personal finance and wealth management app written in TypeScript, with 4,097 stars and 278 forks. It gained 1,320 stars today.
2. Virtual Whiteboard Sketching
    - [excalidraw/excalidraw](https://github.com/excalidraw/excalidraw): A virtual whiteboard for sketching hand-drawn like diagrams, also written in TypeScript. It has 64,113 stars and 5,706 forks, with 268 new stars today.
3. Magical Shell History
    - [atuinsh/atuin](https://github.com/atuinsh/atuin)

# Trigger

Trigger即触发器，代表了OSSWatcher角色的执行时机，最简单的触发方式即定时触发。

以下提供一个定时Trigger的实现，要达到定时的目的，我们可以直接使用asyncio.sleep

In [6]:
import asyncio
import time

from datetime import datetime, timedelta
from metagpt.schema import Message
from pydantic import BaseModel, Field


class OssInfo(BaseModel):
    url: str
    timestamp: float = Field(default_factory=time.time)


async def oss_trigger(hour: int, minute: int, second: int = 0, url: str = "https://github.com/trending"):
    while True:
        now = datetime.now()
        next_time = datetime(now.year, now.month, now.day, hour, minute, second)
        if next_time < now:
            next_time = next_time + timedelta(1)
        wait = next_time - now
        print(wait.total_seconds())
        await asyncio.sleep(wait.total_seconds())
        yield Message(url, OssInfo(url=url))

In [7]:
# 使用aiocron我们可以直接使用cron的语法制定定时任务
# 结合aiocron使用类的方式，来实现定时Trigger

import time
from aiocron import crontab
from typing import Optional
from pytz import BaseTzInfo
from pydantic import BaseModel, Field
from metagpt.schema import Message


class OssInfo(BaseModel):
    url: str
    timestamp: float = Field(default_factory=time.time)


class GithubTrendingCronTrigger():

    def __init__(self, spec: str, tz: Optional[BaseTzInfo] = None, url: str = "https://github.com/trending") -> None:
        self.crontab = crontab(spec, tz=tz)
        self.url = url

    def __aiter__(self):
        return self

    async def __anext__(self):
        await self.crontab.next()
        return Message(self.url, OssInfo(url=self.url))


In [ ]:
# 创建 GithubTrendingCronTrigger 实例，指定每天 UTC 时间 10:00 AM 触发
cron_trigger = GithubTrendingCronTrigger("0 10 * * *")

In [ ]:
# 指定北京时间上午8:00来触发这个任务

from pytz import timezone
beijing_tz = timezone('Asia/Shanghai')  获取北京时间的时区
cron_trigger = GithubTrendingCronTrigger("0 8 * * *", tz=beijing_tz)

In [26]:
current_time = datetime.now()
target_time = current_time + timedelta(minutes=1)
cron_expression = target_time.strftime('%M %H %d %m %w')
# spec = cron_expression   #动态计算cron表达式为运行时当前时间+1分钟，可以根据自己的情况改target_time = current_time + timedelta(minutes=1) 括号里面的数据
cron_expression



'18 17 14 01 0'

In [33]:
from pytz import timezone
## test

async def test_cron_trigger(cron_trigger):
    async for message in cron_trigger:
        print(message)
        break  # 为了测试，我们在接收到第一个消息后就退出循环



current_time = datetime.now()
target_time = current_time + timedelta(minutes=1)
cron_expression = target_time.strftime('%M %H %d %m %w')
print(cron_expression)

# trigger_after 1 min
beijing_tz = timezone('Asia/Shanghai')  #获取北京时间的时区
cron_trigger = GithubTrendingCronTrigger(cron_expression, tz=beijing_tz)

# 启动异步测试
await test_cron_trigger(cron_trigger)

29 17 14 01 0
user: https://github.com/trending


# Callback

### Discord

In [8]:
async def send_discord_msg(channel_id: int, msg: str, token: str):
    intents = discord.Intents.default()
    intents.message_content = True
    client = discord.Client(intents=intents)
    async with client:
        await client.login(token)
        channel = await client.fetch_channel(channel_id)
        await channel.send(msg)

In [9]:
import os
from environs import load_dotenv

load_dotenv('.env')

TOKEN = os.getenv('DISCORD_TOKEN')
CHANNEL_ID = os.getenv('DISCORD_CHANNEL_ID')
proxy = os.getenv('global_proxy')

In [21]:
# callback
import os
import discord
async def discord_callback(msg: Message):
    intents = discord.Intents.default()
    intents.message_content = True
    intents.members = True

    client = discord.Client(intents=intents, proxy=proxy)
    token = os.environ["DISCORD_TOKEN"]
    channel_id = int(os.environ["DISCORD_CHANNEL_ID"])

    async with client:
        await client.login(token)
        channel = await client.fetch_channel(channel_id)
        lines = []
        for i in msg.content.splitlines():
            if i.startswith(("# ", "## ", "### ")):
                if lines:
                    await channel.send("\n".join(lines))
                    lines = []
            lines.append(i)

        if lines:
            await channel.send("\n".join(lines))

In [24]:
from metagpt.subscription import SubscriptionRunner
# 运行入口，
async def main(spec: str = "54 16 * * *", discord: bool = True, wxpusher: bool = False):
    callbacks = []
    if discord:
        callbacks.append(discord_callback)

    if wxpusher:
        callbacks.append(wxpusher_callback)

    if not callbacks:
        async def _print(msg: Message):
            print(msg.content)
        callbacks.append(_print)

    async def callback(msg):
        await asyncio.gather(*(call(msg) for call in callbacks))

    runner = SubscriptionRunner()
    await runner.subscribe(OssWatcher(), GithubTrendingCronTrigger(spec), callback)
    await runner.run()

In [25]:
await main()

2024-01-15 16:54:00.015 | INFO     | __main__:_act:20 - Codey(OssWatcher): ready to CrawlOSSTrending
2024-01-15 16:54:02.677 | INFO     | __main__:_act:20 - Codey(OssWatcher): ready to AnalysisOSSTrending


# Today's GitHub Trending Analysis Report

## Today's Trends
Today, TypeScript, Python, and C continue to dominate as the most popular programming languages. Key areas of interest include AI chatbots, financial management apps, and open-source YouTube apps. The top popular projects are 'EpicGames/raddebugger' and 'danny-avila/LibreChat'.

## The Trends Categories
1. AI Chatbots
    - [LibreChat](https://github.com/danny-avila/LibreChat): An enhanced ChatGPT Clone with 5,549 stars and 209 stars today, written in TypeScript.
    - [open-interpreter](https://github.com/KillianLucas/open-interpreter): A natural language interface for computers with 37,720 stars and 160 stars today, written in Python.
2. Financial Management Apps
    - [maybe](https://github.com/maybe-finance/maybe): A personal finance and wealth management app with 5,867 stars and 2,803 stars today, written in TypeScript.
    - [actual](https://github.com/actualbudget/actual): A local-first personal finance app with 9,922 

CancelledError: 